# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [7]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [10]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])
#x0 = lambda x : x

6

In [13]:
#конкатенация
foldr(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [6]:
def foldl2(f, x0, lst):
    return foldr(..., ..., lst)(...)

In [7]:
def foldr2(f, x0, lst):
    return foldl(..., ..., lst)(...)

In [8]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [46]:
from itertools import permutations

def check_inv(a, b):
    bperms = {str(''.join(item)) for item in permutations(b, len(b))}
    aset = {a[i:(len(b)+i)] for i in range(len(a) - len(b) + 1)}
    return bool(bperms & aset)

a = 'abcrotm'
b = 'tro'
check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [15]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
        if not self.left and not self.right:
            yield self.value
    
    def __str__(self):
        return str([self.value, self.left, self.right])
    
    def __repr__(self):
        return f'value - {self.value}, left - {self.left}, right - {self.right}'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [81]:
def calc(expr):
    return postfix_eval(shunting_yard(expr))

calc('2 * (15 - 3 * 4) - 2') == 4

True

In [75]:

def shunting_yard(expr):
    prior = {'+' : 1, '-' : 1, '*' : 2, '/' : 2}
    #преобразуем expr к удобному виду: ['25', '*', '(', '1545', '-', '3', ')']
    expr = list(''.join(str(expr).split()))
    #print(expr)
    i = 0
    while(True):
        if(expr[i].isdigit() and i < len(expr) and expr[i+1].isdigit()):
            expr[i] += expr[i+1]
            expr.pop(i+1)
        else:
            i += 1
        if i == len(expr)-1:
            break
        
    ans = []
    stack = []
    
    for i in range(len(expr)):
        if expr[i].isdigit():
            ans.append(expr[i])
            continue
        if expr[i] in ['+', '-', '*', '/']:
            while stack and stack[-1] in ['+', '-', '*', '/'] \
            and prior[stack[-1]] >= prior[expr[i]]:
                ans.append(stack.pop())
            stack.append(expr[i])
            continue
        if expr[i] == '(':
            stack.append(expr[i])
            continue
        if expr[i] == ')':
            if not stack:
                print('a parenthesis is omitted in the expression.')
                return
            else:
                while stack[-1] != '(':
                    ans.append(stack.pop())
                    if not stack:
                        print('a parenthesis is omitted in the expression.')
                        return
            stack.pop()
    if stack:
        for i in range(len(stack)-1, -1, -1):
            if stack[i] in ['(', ')']:
                print('a parenthesis is omitted in the expression.')
                return
            ans.append(stack.pop())
    return ans

shunting_yard('25 * (1545 - 3 ) +32/ (2+4)')

['25', '1545', '3', '-', '*', '32', '2', '4', '+', '/', '+']

In [80]:
import operator
ops = { "+": operator.add, "-": operator.sub, \
       "*": operator.mul, "/": operator.truediv}

def postfix_eval(expr):
    stack = []
    for token in expr:
        if token in ['+', '-', '*', '/']:
            oper2 = stack.pop()
            oper1 = stack.pop()
            result = ops[token](float(oper1), float(oper2))
            stack.append(result)
        else:
            stack.append(token)
    return stack.pop()
            
postfix_eval(shunting_yard('2 + (3 / 2)*4'))

8.0